In [1]:
from datasets import load_dataset
import pandas as pd
from feature.selector import Selective, SelectionMethod
from textwiser import TextWiser, Embedding, Transformation
import datasets
import copy
import itertools

In [4]:
dataset = datasets.load_dataset("takojunior/llama_2_finetune", split="train", keep_default_na=False)
    

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [5]:
df = pd.DataFrame(dataset)
df.head()

,instruction,input,output
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...
1,Which is a species of fish? Tope or Rope,,Tope
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981."


In [6]:
df['prompt'] = df['instruction'] + " " + df['input']
df.head()

,instruction,input,output,prompt
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,When did Virgin Australia start operating? Vir...
1,Which is a species of fish? Tope or Rope,,Tope,Which is a species of fish? Tope or Rope
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,Why can camels survive for long without water?
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,"Alice's parents have three daughters: Amy, Jes..."
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",When was Tomoaki Komorida born? Komorida was b...


In [7]:
df['prompt_tokens'] = df['prompt'].apply(lambda x: len(x.split()))
df['response_tokens'] = df['output'].apply(lambda x: len(x.split()))
df.head()

,instruction,input,output,prompt,prompt_tokens,response_tokens
0,When did Virgin Australia start operating?,"Virgin Australia, the trading name of Virgin A...",Virgin Australia commenced services on 31 Augu...,When did Virgin Australia start operating? Vir...,91,18
1,Which is a species of fish? Tope or Rope,,Tope,Which is a species of fish? Tope or Rope,9,1
2,Why can camels survive for long without water?,,Camels use the fat in their humps to keep them...,Why can camels survive for long without water?,8,20
3,"Alice's parents have three daughters: Amy, Jes...",,The name of the third daughter is Alice,"Alice's parents have three daughters: Amy, Jes...",15,8
4,When was Tomoaki Komorida born?,Komorida was born in Kumamoto Prefecture on Ju...,"Tomoaki Komorida was born on July 10,1981.",When was Tomoaki Komorida born? Komorida was b...,223,7


In [8]:
df['prompt_tokens'].describe()

count    600210.000000
mean        380.991726
std         446.955032
min           1.000000
25%          66.000000
50%         209.000000
75%         519.000000
max       25384.000000
Name: prompt_tokens, dtype: float64

In [13]:
df['prompt_tokens'].quantile(q=0.9)

1090.0

In [9]:
df['response_tokens'].describe()

count    600210.000000
mean         21.499339
std         144.128741
min           0.000000
25%           1.000000
50%           2.000000
75%          10.000000
max       32367.000000
Name: response_tokens, dtype: float64

In [11]:
df['response_tokens'].quantile(q=0.9)

34.0